# Usage suggestions for this python package

First install the package:

Activate the python environment you want to use. Go into the folder of this notebook, where setup.py located is, and run 

`pip install -e .`

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import relational_image_generation_evaluation as rige

In [2]:
evaluator = rige.Evaluator('GraphCLIP')
# testonly just makes loading faster, to actually test the model you need to set testonly=False
# this is not the test set or so. It's just a flag to make loading faster
dataloader_one = rige.get_one_edge_dataloader(testonly=True)
dataloader_two = rige.get_two_edge_dataloader(testonly=True)
print("len(dataloader_one):", len(dataloader_one))
print("len(dataloader_two):", len(dataloader_two))

Using device cuda:0 for evaluation.
Loading filtered test graphs...
Finished loading filtered test graphs
Generating one edge graphs...


100%|██████████| 100/100 [00:00<00:00, 1030.45it/s]

Finished generating one edge graphs
Loading filtered test graphs...


Finished loading filtered test graphs
Generating two edge graphs...


100%|██████████| 100/100 [00:00<00:00, 777.73it/s]

Finished generating two edge graphs
len(dataloader_one): 837
len(dataloader_two): 1076


**The dataloaders give you networkx graphs as they have convenient functionality.**

**You can access all scene information like this (images have to be loaded using PATH_TO_VG/image_id.jpg):**

In [3]:
some_graph = next(iter(dataloader_one))[0]
print("nodes:", some_graph.nodes)
print("edges:", some_graph.edges)
edge = list(some_graph.edges)[0]
print("some edge:", edge, "edge predicate:", some_graph.edges[edge]['predicate'])
node = list(some_graph.nodes)[0]
print("some node:", node, "node label:", some_graph.nodes[node]['name'])
# other information:
print(some_graph.nodes[node].keys())
print(some_graph.edges[edge].keys())
print("image_id:", some_graph.image_id)
print("image dimensions:", some_graph.image_w, some_graph.image_h)
# use this copy function to make a copy of the graph, because much faster than deepcopy and normal copy doesn't copy everything
some_graph_copy = rige.copy_graph(some_graph)

nodes: [3632782, 1634343]
edges: [(1634343, 3632782)]
some edge: (1634343, 3632782) edge predicate: on
some node: 3632782 node label: street
dict_keys(['w', 'h', 'x', 'y', 'attributes', 'name'])
dict_keys(['synsets', 'relationship_id', 'predicate'])
image_id: 57
image dimensions: 800 600


In [4]:
some_graph.nodes

NodeView((3632782, 1634343))

In [5]:
from PIL import Image
import os
images = []
graphs = []
for i in range(10):
    graph = next(iter(dataloader_one))[0]
    image_id = graph.image_id
    # adapt to your local directory
    IMAGE_DIR = '../datasets/visual_genome/raw/VG_100K/'
    if not os.path.exists(IMAGE_DIR + str(image_id) + '.jpg'):
        IMAGE_DIR = '../datasets/visual_genome/raw/VG_100K_2/'
    path = IMAGE_DIR + str(image_id) + '.jpg'
    image = Image.open(path)
    images.append(image)
    graphs.append(graph)

The ViT finetuned models: For each graph you evaluate they give you a relationship probability confidence and an attribute probability confidence. Both are the mean confidences for the relationships and attributes of the graph you give in.

In many graphs, there are no attributes, in this case, 'noattributes' os given back instead of a numerical score. Same behaviour for edges.

In [13]:
scores = evaluator(images,graphs)
print(scores)
# print(scores.keys())

 30%|███       | 3/10 [00:00<00:00, 25.72it/s]

graph DiGraph with 2 nodes and 1 edges
nx DiGraph with 2 nodes and 1 edges
d {'objects': [{'names': ['sign'], 'attributes': ['white'], 'object_id': 1062341}, {'names': ['writing'], 'attributes': [], 'object_id': 1062362}], 'relationships': [{'object_id': 1062362, 'subject_id': 1062341, 'predicate': 'has'}]}
graph DiGraph with 2 nodes and 1 edges
nx DiGraph with 2 nodes and 1 edges
d {'objects': [{'names': ['building'], 'attributes': ['red', 'brick'], 'object_id': 1534698}, {'names': ['window'], 'attributes': [], 'object_id': 1534701}], 'relationships': [{'object_id': 1534698, 'subject_id': 1534701, 'predicate': 'on'}]}
graph DiGraph with 2 nodes and 1 edges
nx DiGraph with 2 nodes and 1 edges
d {'objects': [{'names': ['wall'], 'attributes': ['white'], 'object_id': 1061746}, {'names': ['legs'], 'attributes': ['silver', 'long'], 'object_id': 5975}], 'relationships': [{'object_id': 5975, 'subject_id': 1061746, 'predicate': 'has'}]}
graph DiGraph with 2 nodes and 1 edges
nx DiGraph with 2 

100%|██████████| 10/10 [00:00<00:00, 38.79it/s]


graph DiGraph with 2 nodes and 1 edges
nx DiGraph with 2 nodes and 1 edges
d {'objects': [{'names': ['car'], 'attributes': ['black'], 'object_id': 1025395}, {'names': ['road'], 'attributes': ['wet'], 'object_id': 1025403}], 'relationships': [{'object_id': 1025403, 'subject_id': 1025395, 'predicate': 'on'}]}
graph DiGraph with 2 nodes and 1 edges
nx DiGraph with 2 nodes and 1 edges
d {'objects': [{'names': ['cabinet'], 'attributes': [], 'object_id': 1534237}, {'names': ['door'], 'attributes': [], 'object_id': 1534236}], 'relationships': [{'object_id': 1534237, 'subject_id': 1534236, 'predicate': 'on'}]}
graph DiGraph with 2 nodes and 1 edges
nx DiGraph with 2 nodes and 1 edges
d {'objects': [{'names': ['chair'], 'attributes': ['brown', 'wood'], 'object_id': 1060348}, {'names': ['desk'], 'attributes': ['brown'], 'object_id': 1060368}], 'relationships': [{'object_id': 1060368, 'subject_id': 1060348, 'predicate': 'near'}]}


100%|██████████| 10/10 [00:21<00:00,  2.14s/it]

[[0.12640926241874695, 0.017500873655080795, -0.06303272396326065, -0.028431672602891922, 0.20559445023536682, -0.01714608445763588, 0.671425998210907, 0.5806416869163513, 0.04830455780029297, -1.8447648286819458, 0.018201563507318497, -0.04191529005765915, 0.44231000542640686, 0.6538582444190979, 0.0038655931130051613, -0.18623259663581848, 0.10422366857528687, 0.03902389109134674, 0.006715813651680946, 0.020725596696138382, 0.1340392380952835, 0.06050331890583038, 0.0013882210478186607, 0.20208168029785156, 0.15641173720359802, 0.0021357256919145584, -0.014975237660109997, -0.16900448501110077, -0.02959018386900425, 0.000132763379951939, -0.6284950971603394, 0.21447251737117767, 0.07660213112831116, 0.006224311422556639, -0.205814927816391, -0.044392094016075134, -0.004012417979538441, 0.016870275139808655, -0.033157236874103546, -0.3868199586868286, 0.1027725338935852, 0.3449122905731201, 0.010984734632074833, -0.07476146519184113, 0.013024985790252686, 0.009209156036376953, 0.02685

# Use the Human Preference Score

In [6]:
HPS = rige.HumanPreferenceScore()

Downloading...
From (uriginal): https://drive.google.com/uc?id=1_KL-3i4CbDgKAAjvZu0qiDYd0Q0i1o0Z
From (redirected): https://drive.google.com/uc?id=1_KL-3i4CbDgKAAjvZu0qiDYd0Q0i1o0Z&confirm=t&uuid=23fa1dbb-e8bf-468d-8c1b-ccbdb9b67e7c
To: /local/home/jthomm/GraphCLIP/relational_image_generation_evaluation/relational_image_generation_evaluation/data/hpc.pt
100%|██████████| 2.91G/2.91G [00:23<00:00, 126MB/s] 


Downloaded weights for the HumanPreferenceScore Model to /local/home/jthomm/GraphCLIP/relational_image_generation_evaluation/relational_image_generation_evaluation/data/hpc.pt


In [7]:
print(HPS.compute("a red car", [images[0], images[1]]))
print(HPS.compute_from_paths("a red car", [path, path]))

tensor([[0.1696],
        [0.1660]], device='cuda:2', dtype=torch.float16)
tensor([[0.1776],
        [0.1776]], device='cuda:2', dtype=torch.float16)
